## Try DuckDB

[https://duckdb.org](https://duckdb.org)

In [5]:
import sys
sys.path

['/Users/mjboothaus/code/github/databooth/try-duckdb',
 '/Users/mjboothaus/.vscode/extensions/ms-toolsai.jupyter-2021.8.1195043623/pythonFiles',
 '/Users/mjboothaus/.vscode/extensions/ms-toolsai.jupyter-2021.8.1195043623/pythonFiles/lib/python',
 '/opt/homebrew/Caskroom/mambaforge/base/envs/emmaus_walking_py38/lib/python39.zip',
 '/opt/homebrew/Caskroom/mambaforge/base/envs/emmaus_walking_py38/lib/python3.9',
 '/opt/homebrew/Caskroom/mambaforge/base/envs/emmaus_walking_py38/lib/python3.9/lib-dynload',
 '',
 '/opt/homebrew/Caskroom/mambaforge/base/envs/emmaus_walking_py38/lib/python3.9/site-packages',
 '/opt/homebrew/Caskroom/mambaforge/base/envs/emmaus_walking_py38/lib/python3.9/site-packages/IPython/extensions',
 '/Users/mjboothaus/.ipython']

In [6]:
import duckdb

ModuleNotFoundError: No module named 'duckdb'

In [3]:
cursor = duckdb.connect()
print(cursor.execute('SELECT 42').fetchall())

[(42,)]


In [5]:
import pandas as pd

In [8]:
mydf = pd.DataFrame({'a' : [1, 2, 3]})
print(duckdb.query("SELECT AVG(a) FROM mydf").to_df())

   avg(a)
0     2.0


In [7]:
!jupyter kernelspec list

Available kernels:
  python3810jvsc74a57bd03086ab3d90372c825407b804b6256634f337f6e28cf9335d0f6d2a4e138be96d    /Users/mjboothaus/Library/Jupyter/kernels/python3810jvsc74a57bd03086ab3d90372c825407b804b6256634f337f6e28cf9335d0f6d2a4e138be96d
  python3810jvsc74a57bd099143769d1b812eb68db2dcb5b68ac9d4e914bf3cc41efab59d780dae4e055d1    /Users/mjboothaus/Library/Jupyter/kernels/python3810jvsc74a57bd099143769d1b812eb68db2dcb5b68ac9d4e914bf3cc41efab59d780dae4e055d1
  python3810jvsc74a57bd0cfa7a8ce30da2c99891659cad67a24848f787c9950064e6e35407c1189265faa    /Users/mjboothaus/Library/Jupyter/kernels/python3810jvsc74a57bd0cfa7a8ce30da2c99891659cad67a24848f787c9950064e6e35407c1189265faa
  python3810jvsc74a57bd0f240be019856415b64564560067a995388062410618151689d60bca2458eef6d    /Users/mjboothaus/Library/Jupyter/kernels/python3810jvsc74a57bd0f240be019856415b64564560067a995388062410618151689d60bca2458eef6d
  python3810jvsc74a57bd0fec88cd268bab3422d222fe82790b66e2cd0da331467ceb1b83c1e5368aa9d0e    /User